Имеем абстрактную алгебру Ли, заданную следующими базовыми коммутаторами:
* $[S_1, S_2] = -\frac{ih}{2}(S_0 S_3 + S_3 S_0)$
* $[S_2, S_3] = \frac{ih}{2}(S_0 S_1 + S_1 S_0)$
* $[S_3, S_1] = \frac{ih}{2}(S_0 S_2 + S_2 S_0)$


* $[S_0, S_1] = -2ihS_2$
* $[S_0, S_2] = 2ihS_1$
* $[S_0, S_3] = 0$

## Задание 1

Зададим структуру "рождение-уничтожение" по формулам
* $C = S_1 + iS_2$
* $B = S_1 - iS_2$
* $A_1 = S_0$
* $A_2 = S_3$

In [1]:
import sympy as sp
from sympy import I
from sympy.physics.quantum import Commutator, Operator

sp.init_printing()

# Зададим операторы
S0, S1, S2, S3 = list(map(Operator, ['S_0', 'S_1', 'S_2', 'S_3']))
A1, A2, C, B = list(map(Operator, ['A_1', 'A_2', 'C', 'B']))
h = sp.Symbol("h")

# Зададим входную алгебру
commutator_map = {
    Commutator(S1, S2): -I * h / 2 * (S0 * S3 + S3 * S0),
    Commutator(S2, S3): I * h / 2 * (S0 * S1 + S1 * S0),
    Commutator(S3, S1): I * h / 2 * (S0 * S2 + S2 * S0),
    Commutator(S0, S1): -2 * h * I * S2,
    Commutator(S0, S2): 2 * h * I * S1,
    Commutator(S0, S3): 0
}

to_new_algebra = {
    S0: A1,
    S3: A2,
    S1 + I * S2: C,
    S1 - I * S2: B
}

to_old_algebra = dict((v, k) for k, v in to_new_algebra.items())

### 1. Вычислим $[C, B]$

In [2]:
Commutator(C, B)\
    .subs(to_old_algebra)\
    .expand(commutator=True).expand(commutator=True)\
    .subs(commutator_map)\
    .factor()\
    .subs(to_new_algebra)

$[C, B] = -h(A_1 A_2 + A_2 A_1)$

### 2. Найдём $\phi(A)$ из выражений $CA = \phi(A) C$, $AB = B \phi(A)$

Как вариант, найдем $\phi$ из второго равенства.
В ходе дальнейшей проверки, найденная функция $\phi$ подтвердилась и для первого равенства.

#### 2.1. Найдём $\phi_1(A)$
Воспользуемся коммутатором $[A_1, B] = A_1 B - B A_1$. Тогда $A_1 B = [A_1, B] + B A_1$

Сейчас вычислим коммутатор.

In [3]:
Commutator(A1, B)\
    .subs(to_old_algebra)\
    .expand(commutator=True).expand(commutator=True)\
    .subs(commutator_map)\
    .factor()\
    .subs(to_new_algebra)

Получили $A_1 B = 2hB + B A_1 = B(2h + A_1)$

$\phi_1(A) = 2h + A_1$

Несложно убедиться, что для в формуле с $C$ получим то же самое.

In [4]:
Commutator(C, A1)\
    .subs(to_old_algebra)\
    .expand(commutator=True).expand(commutator=True)\
    .subs(commutator_map)\
    .factor()\
    .subs(to_new_algebra)

#### 2.2. Найдём $\phi_2(A)$

Воспользуемся коммутатором $[A_2, B] = A_2 B - B A_2$. Тогда $A_2 B = [A_2, B] + B A_2$

Вычислим коммутатор.

In [5]:
Commutator(A2, B)\
    .subs(to_old_algebra)\
    .expand(commutator=True).expand(commutator=True)\
    .subs(commutator_map)\
    .cancel()\
    .apart()\
    .subs(to_old_algebra[B], B)

Доведём дело до конца:
$[A_2, B] = -\frac{h}{2}\Big(S_0 (S_1 - iS_2) + (S_1 - iS_2)S_0\Big) = -\frac{h}{2}\Big(A_1 B + B A_1\Big)$

$A_1 B$ мы уже вычисляли, и, в итоге получим

$A_2 B = B(A_2 - hA_1 - h^2)$

$\phi_2(A) = A_2 - hA_1 - h^2$

### 3. Доказательство $[A_{\mu}, A_{\nu}]$ тривиально.

## Задание 2

Найти оператор Казимира $K$

**Теорема**
1. Если $\rho(\phi(A)) - \rho(A) = f(A)$, то $\hat K = \hat B \hat C - \rho(\hat A)$
2. Если $\kappa(\phi(A)) = \kappa(A),\ len(A) > 1$, то $\hat \kappa = \kappa(\hat A)$ - оператор Казимира.

#### 1. Поиск в виде $\kappa(A)$

В начале подбора $\kappa$ в виде полиномиальной функции с помощью метода неопределённых коэффициентов выяснилось, что
$\kappa(A_1, A_2) = A_1 A_2 - A_2 A_1 = [A_1, A_2]$

Действительно:

In [6]:
def phi(A1, A2):
    return (A1 + 2*h, A2 - h*A1 - h**2)


def kappa(A1, A2):
    return A1 * A2 - A2 * A1


kappa(*phi(A1, A2)).expand() - kappa(A1, A2)

Тогда оператор Казимира: $\hat{\kappa} = \kappa({\hat A})$

#### 1. Поиск в виде $\rho(A)$

Заметим, что квадрат $\phi_2(A)$ даёт нам в компонентах искомое значение $f(A) = -h(A_1 A_2 + A_2 A_1)$

In [7]:
((A2 - h*A1 - h**2)**2).expand()

Поэтому, попробуем избавиться от остальных компонентов в данном выражении.
Для этого предположим, что $\rho(A)$ - полином следующего вида

$$\rho(A) = a_1 A_1^2 + A_2^2 + a_3 A_1 + a_4 A_2 + a_5 A_1 A_2 + a_6 A_2 A_1$$



In [8]:
a1, a2, a3, a4, a5, a6, a7 = sp.symbols(",".join([f'a_{i}' for i in range(1, 8)]), real=True)


def rho(A1, A2):
    return a1 * A1**2 + a2 * A1 * A2 + a3 * A2 * A1 + a4*A2**2 + a5 * A1 + a6 * A2 + a7

def rho2(A1, A2):
    return a1*A1**2 + A2**2 + a3 * A1 + a4*A2 + a5*A1*A2 + a6*A2*A1

rho2(A1, A2)


Выпишем $\rho(\phi(A)) - \rho(A)$:

In [9]:
(rho2(*phi(A1, A2)) - rho2(A1, A2)).expand()

Далее, мы решили систему линейных уравнений относительно мономов, получив тем самым ограничения на коэффициенты $a_i$:
* $a_5 + a_6 = h$
* $a_4 = 4a_1 - h^2$
* $a_3 = 0$

Подставим их конкретные значения в $\rho(\phi(A)) - \rho(A)$:

In [10]:
(rho2(*phi(A1, A2)) - rho2(A1, A2)).expand()\
    .subs({a1: 1, a3: 0, a4: 4 - h**2, a5: 0, a6: h}).expand()

Получили, что $\rho$ задаётся параметрическим семейством полиномов:

$\rho(A) = A_2^2 + b A_1^2+ a A_1 A_2 + (1 - a) A_2 A_1 + (4b - h^2) A_2, \quad a, b \in \mathbb{R}$

В итоге, получим оператор Казимира как $\hat K = \hat B \hat C - \rho(\hat A)$



## Задание 3

Найти спектр коммутативной подалгебры, вычислив функции $\mathbb{A}_a(n)$ и $F_a(n)$

#### 1. Вычисление $\mathbb{A}_a(n)$

$\mathbb{A}_a(n) = \phi^n(a)$

Попробуем рассмотреть структуру функции при различных $n$.

In [11]:
def phi_n(A1, A2, n):
    a, b = phi(A1, A2)
    if n <= 1:
        return a, b.expand()
    return phi_n(a, b, n-1)

a, b = phi_n(A1, A2, 1)
a, b

In [12]:
a, b = phi_n(A1, A2, 2)
a, b


In [13]:
a, b = phi_n(A1, A2, 3)
a, b

In [14]:
a, b = phi_n(A1, A2, 4)
a, b

Подозрение падает на следующую структуру функции:

$\mathbb{A}_a(n) = (A_1 + 2nh,\quad A_2 - nhA_1 - n^2h^2)$

Если с большей частью коэффициентов всё очевидно, то член при $h^2$ задаётся следующим рекуррентным соотношением:
$a_{n + 1} = a_n + 2n + 1$. Решим его.

1. Характеристический многочлен однородного уравнения: $\lambda - 1 = 0 \implies \lambda = 1 \implies \bar a_n = G \cdot 1^n$
2. Частное решение: $a_n = n(c_0 + c_1n) \xrightarrow[]{n \to \ n + 1} 2c_1 n + c_1 + c_0 = 2n + 1$
   * $c_1 = 1$
   * $c_0 = -2$
   * $\tilde a_n = n^2 - 2$
3. $a_n + G + n^2 - 2 \xrightarrow[]{a_1 = 1}  2 + n^2 - 2 = n^2$

Таким образом, уравнение для $\mathbb{A}_a(n)$ верно.


#### 2. Вычисление $F_a(n)$

$F_a(n) = \frac{1}{n + 1}\sum_{j = 0}^n f(\mathbb{A}_a(j))$

Вычислим, для начала $f(\mathbb{A})$

In [15]:
n = sp.Symbol("n", integer=True, positive=True)

A_a = A1 + 2*n*h, A2 - n*h*A1 - n**2 * h**2
A_a

In [16]:
def subs_n(v):
    a, b = A_a
    return a.subs(n, v), b.subs(n, v)

def f(A1, A2):
    return -h * (A1 * A2 + A2 * A1)

fn = f(*A_a).simplify().expand().simplify()
fn

Перепишем для j:

In [19]:
j = sp.Symbol("j", integer=True, positive=True)
fj = fn.subs(n, j)
fj.expand()

Как мы видим, сумма $F_a(n) = \frac{1}{n + 1}\sum_{j = 0}^n f(\mathbb{A}_a(j))$ разбивается на много тривиальных сумм:

$F_a(n) = \frac{h}{n + 1}\Big(4h^3 \sum_j j^3 + 6h^2A_1 \sum_j j^2 + 2hA_1^2 \sum_j j - 4hA_2 \sum_j j - n(A_1 A_2 + A_2 A_1)  \Big)$

* $\sum_{j = 0}^m j^3 = \frac{1}{4}n^2(n+1)^2$
* $\sum_{j = 0}^n j^2 = \frac{1}{6}n(n + 1)(2n + 1)$
* $\sum_{j = 0}^n j = \frac{1}{2}n(n + 1)$

$F_a(n) = \frac{h}{n + 1}\Big(h^3n^2(n+1)^2 + h^2 n(n + 1)(2n + 1) A_1  + h n(n + 1) A_1^2 - 2hn(n + 1)A_2 - n(A_1 A_2 + A_2 A_1)  \Big)$

Раскроем внешнюю скобку

$F_a(n) = h^4n^2(n+1) + h^3 n(2n + 1) A_1  + h^2 n A_1^2 - 2h^2nA_2 - \frac{nh}{n + 1}(A_1 A_2 + A_2 A_1)$
